## GDM PROJECT 2 

BY: ABDULMALIK AJIBADE

In [1]:
import pandas as pd 
import numpy as np 
import geopandas as gpd
import matplotlib.pyplot as plt
import sqlalchemy

## Project Tasks

### 1. Creation of the first SHP (instruments):

- Open the 4 files with pandas.
- Merge it in a new DataFrame.
- Make a GeoDataFrame with all the instruments (convert geometry from EPSG:4978 to EPSG:4326).
- Calculate the site_id and the instrument_id (add new columns).
- Remove useless columns.
- Save the data!

In [2]:
inst_1 = pd.read_fwf('ITRF2020_DORIS_cart.txt')
inst_2 = pd.read_fwf('ITRF2020_GNSS_cart.txt')
inst_3 = pd.read_fwf('ITRF2020_SLR_cart.txt')
inst_4 = pd.read_fwf('ITRF2020_VLBI_cart.txt')

In [3]:
instrument_data = pd.concat([inst_1, inst_2, inst_3, inst_4])

In [4]:
instrument_data.head()

,id,name,type,code,x,y,z,dx,dy,dz
0,10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,556166.4818,4.389372e+06,0.002,0.0025,0.002
1,10002S019,Grasse (OCA),DORIS,GR4B,4.581681e+06,556166.9141,4.389371e+06,0.0019,0.0024,0.0017
2,10003S001,Toulouse,DORIS,TLSA,4.628047e+06,119670.6873,4.372788e+06,0.0054,0.0062,0.0051
3,10003S003,Toulouse,DORIS,TLHA,4.628693e+06,119985.0770,4.372105e+06,0.0034,0.0042,0.0032
4,10003S005,Toulouse,DORIS,TLSB,4.628694e+06,119985.0787,4.372105e+06,0.0026,0.0039,0.0025


In [5]:
instrument_gdf = gpd.GeoDataFrame(gpd.GeoDataFrame(instrument_data, 
                                                   geometry=gpd.points_from_xy(instrument_data['x'], 
                                                                               instrument_data['y'], 
                                                                               instrument_data['z']), 
                                                                              crs="EPSG:4978" )

)


instrument_gdf = instrument_gdf.to_crs(epsg=4326)


In [6]:
instrument_gdf.head()

,id,name,type,code,x,y,z,dx,dy,dz,geometry
0,10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,556166.4818,4.389372e+06,0.002,0.0025,0.002,POINT Z (6.92123 43.75483 1323.70087)
1,10002S019,Grasse (OCA),DORIS,GR4B,4.581681e+06,556166.9141,4.389371e+06,0.0019,0.0024,0.0017,POINT Z (6.92123 43.75483 1323.8158)
2,10003S001,Toulouse,DORIS,TLSA,4.628047e+06,119670.6873,4.372788e+06,0.0054,0.0062,0.0051,POINT Z (1.48121 43.55814 207.69101)
3,10003S003,Toulouse,DORIS,TLHA,4.628693e+06,119985.0770,4.372105e+06,0.0034,0.0042,0.0032,POINT Z (1.48489 43.54962 210.79597)
4,10003S005,Toulouse,DORIS,TLSB,4.628694e+06,119985.0787,4.372105e+06,0.0026,0.0039,0.0025,POINT Z (1.48489 43.54962 211.08413)


In [7]:
instrument_gdf['site_id'] = instrument_gdf['id'].str[:5]
instrument_gdf['instrument_id'] = instrument_gdf['id'].str[5:]

In [8]:
instrument_gdf = instrument_gdf.drop(columns=['dx', 'dy', 'dz'])

In [9]:
instrument_gdf.to_file("instruments.gpkg", layer='layer_name', driver="GPKG")

In [10]:
instrument_gdf.head()

,id,name,type,code,x,y,z,geometry,site_id,instrument_id
0,10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,556166.4818,4.389372e+06,POINT Z (6.92123 43.75483 1323.70087),10002,S018
1,10002S019,Grasse (OCA),DORIS,GR4B,4.581681e+06,556166.9141,4.389371e+06,POINT Z (6.92123 43.75483 1323.8158),10002,S019
2,10003S001,Toulouse,DORIS,TLSA,4.628047e+06,119670.6873,4.372788e+06,POINT Z (1.48121 43.55814 207.69101),10003,S001
3,10003S003,Toulouse,DORIS,TLHA,4.628693e+06,119985.0770,4.372105e+06,POINT Z (1.48489 43.54962 210.79597),10003,S003
4,10003S005,Toulouse,DORIS,TLSB,4.628694e+06,119985.0787,4.372105e+06,POINT Z (1.48489 43.54962 211.08413),10003,S005


### 2. Creation of the second SHP (sites):

- Keep only the instruments that belongs to a site (look at the five first numbers of the DOMES (id) number) which hosts at least 3 instruments from 2 different measurement techniques (GNSS, DORIS, SLR or VLBI).
- Make a spatial groupby (dissolve) two join all the points from a same site.
- Calculate a polygon from the list of points (you will need the shapely Polygon function and the shapely convex_hull property).
- Save the data!

In [11]:


site_stats = (
    instrument_gdf.groupby('site_id')
    .agg(instrument_count=('instrument_id', 'size'),  # Count of instruments
         unique_types=('type', 'nunique'))           # Count of unique measurement techniques
    .reset_index()
)


eligible_sites = site_stats[
    (site_stats['instrument_count'] >= 3) &
    (site_stats['unique_types'] >= 2)
]


filtered_sites = instrument_gdf[instrument_gdf['site_id'].isin(eligible_sites['site_id'])]



In [12]:
site_counts = instrument_gdf['site_id'].value_counts()


instrument_gdf_filtered = instrument_gdf[instrument_gdf['site_id'].\
                                         isin(site_counts[site_counts > 2].index)]

In [13]:
site_counts = instrument_gdf['site_id'].value_counts()


instrument_gdf_filtered = instrument_gdf[instrument_gdf['site_id'].\
                                         isin(site_counts[site_counts > 2].index)]

In [23]:
instruments_dissolved = instrument_gdf_filtered.dissolve(by='site_id')

In [24]:
instruments_dissolved.head()

,geometry,id,name,type,code,x,y,z,instrument_id
site_id,,,,,,,,,
10002,"MULTIPOINT Z (6.92058 43.75474 1319.30335, 6.9...",10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,5.561665e+05,4.389372e+06,S018
10003,"MULTIPOINT Z (1.48076 43.56077 207.09635, 1.48...",10003S001,Toulouse,DORIS,TLSA,4.628047e+06,1.196707e+05,4.372788e+06,S001
10004,"MULTIPOINT Z (-4.50383 48.40787 104.42082, -4....",10004M004,Brest,GNSS,BRST,4.231162e+06,-3.327465e+05,4.745131e+06,M004
10077,"MULTIPOINT Z (8.76246 41.92747 98.24128, 8.762...",10077S002,Ajaccio,DORIS,AJAB,4.696990e+06,7.239812e+05,4.239679e+06,S002
10202,"MULTIPOINT Z (-21.99518 64.15098 95.75501, -21...",10202S001,Reykjavik,DORIS,REYA,2.585528e+06,-1.044368e+06,5.717159e+06,S001


In [25]:
instruments_dissolved['Polygon'] = instruments_dissolved['geometry'].convex_hull

In [26]:
instruments_dissolved.head()

,geometry,id,name,type,code,x,y,z,instrument_id,Polygon
site_id,,,,,,,,,,
10002,"MULTIPOINT Z (6.92058 43.75474 1319.30335, 6.9...",10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,5.561665e+05,4.389372e+06,S018,"POLYGON Z ((6.92077 43.75449 1319.85842, 6.920..."
10003,"MULTIPOINT Z (1.48076 43.56077 207.09635, 1.48...",10003S001,Toulouse,DORIS,TLSA,4.628047e+06,1.196707e+05,4.372788e+06,S001,"POLYGON Z ((1.48489 43.54962 210.79597, 1.4812..."
10004,"MULTIPOINT Z (-4.50383 48.40787 104.42082, -4....",10004M004,Brest,GNSS,BRST,4.231162e+06,-3.327465e+05,4.745131e+06,M004,"POLYGON Z ((-4.49659 48.3805 65.82489, -4.5038..."
10077,"MULTIPOINT Z (8.76246 41.92747 98.24128, 8.762...",10077S002,Ajaccio,DORIS,AJAB,4.696990e+06,7.239812e+05,4.239679e+06,S002,"POLYGON Z ((8.7627 41.92739 96.80211, 8.76246 ..."
10202,"MULTIPOINT Z (-21.99518 64.15098 95.75501, -21...",10202S001,Reykjavik,DORIS,REYA,2.585528e+06,-1.044368e+06,5.717159e+06,S001,"POLYGON Z ((-21.95549 64.13879 93.04831, -21.9..."


In [27]:
instruments_dissolved = instruments_dissolved.drop(columns='geometry')

In [28]:
instruments_dissolved.to_file("instruments_polygon.gpkg", layer='layer_name', driver="GPKG")

### 3. Creation of the last SHP (images):

- For each site, list the images (between 2022/01/01 and 2022/09/30) that are covering the extent of the site. This is very long!!! Write the information in a <site_id>.json temporary file to be able to restart the script if it fails.
- Merge all information in one GeoDataFrame.
- Save the data!

In [30]:
import requests

def request_images(wkt_geometry):
    items = [] # Empty list to store return elements
    # Request
    r = requests.get(
        "https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json",
        params={
            "geometry": wkt_geometry,
            "startDate": "2022-01-01T00:00:00.000Z",
            "completionDate": "2022-09-30T23:59:59.999Z",
            "cloudCover": "[0,10]",
            "maxRecords": 20,
            "page": 1,
        }
    )
    # If status_code is not 200, we have an issue
    if r.status_code == 200:
        data = r.json()
        if 'features' in data:
            items += data['features']
    return items


In [31]:
instruments_dissolved['images'] = instruments_dissolved['Polygon'].apply(lambda x: request_images(x.wkt))

In [44]:
import json


instruments_dissolved.to_csv("instruments_dissolved.csv")

# Save the images information for each site in a JSON file
for site_id, images in instruments_dissolved['images'].items():
    with open(f"{site_id}.json", "w") as f:
        json.dump(images, f)

#### Visualise some images 

In [41]:
# Display images for only 5 sites
for site_id, images in instruments_dissolved['images'].head(5).items():
    if not isinstance(images, list):
        print(f"Site {site_id} does not have a list of images.")
    else:
        for image in images:
            if not isinstance(image, dict):
                print(f"Site {site_id} has an image that is not a dictionary.")
            elif 'type' not in image or 'id' not in image or 'geometry' not in image or 'properties' not in image:
                print(f"Site {site_id} has an image with missing keys.")
            else:
                print(f"Site {site_id} has a valid image with id: {image['id']}")
                if 'thumbnail' in image['properties']:
                    display(Image(url=image['properties']['thumbnail']))


Site 10002 has a valid image with id: 3aaee0d9-5dc8-50f4-ae6a-5627e984ba2a


Site 10002 has a valid image with id: 00951cef-b763-5d0f-8fe8-b7212c7588c5


Site 10002 has a valid image with id: 0eab919b-be5b-575a-a764-c1a9d17f1bbf


Site 10002 has a valid image with id: 4743ad8e-ffc4-5b5f-a5dc-3dc933ddf292


Site 10002 has a valid image with id: 87612d35-a8cd-51cd-85d9-715ebd00f756


...

Site 10077 has a valid image with id: 262c95df-774c-55bd-9d28-5d5c26798b08


Site 10077 has a valid image with id: 5d1a0251-9dda-5ffc-9fdf-6e229038b788


Site 10202 has a valid image with id: ffcb5df4-d22c-545a-99e0-5998f012de85


Site 10202 has a valid image with id: e259bada-8c84-5f55-aeca-e4376eecf5d5


Site 10202 has a valid image with id: a9ef7679-f2b5-5eaa-8de2-2c42e403495a


In [22]:
images = instruments_dissolved['Polygon'].apply(request_images)
# images = request_images(wkt_geometry)